# Machine Learning Engineer Nanodegree

## Capstone project: Exoplanet search (from Kaggle)

Student Name: Graciano Patino

Kaggle reference: https://www.kaggle.com/keplersmachines/kepler-labelled-time-series-data?/

The mission as stated in the Github (https://github.com/winterdelta/KeplerAI) is to build a classification algorithm for identifying if a particular time series input includes an exoplanet or not. It also mentions that a number of methods were tested: 1-D CNN in Torch7, XGBoost in R and PCA in Python. However, none of these methods provided strong results according to the kaggle and Github references. 

For this project, I would evaluate deep learning algorithms. Per paper in the paragraph (below), these algorithms appear to provide better results compared to the ones already tried as mentioned above.

1)	Initially I would evaluate 1-D CNN using Keras instead of Torch7. 
2)	Based on reference paper, I would try adding different number of layers and filters in combination with other CNN parameters. Details would be included in project report.
3)	The output of the CNNs would be the input to one or more dense layers.
4)	Performance of each model to be measured as per evaluation metrics section.
5)	Per kaggle source the test set is confirmed to have 5 exoplanets. This will also be useful on checking performance of algorithms. If an algorithm is unable to identify exoplanets on then testing set, then model might not be good. 

Please that the list above of models considered is not meant to be exhaustive for all possible scenarios in deep learning algorithms. It might be the case that other deep learning algorithms might be considered later should the ones proposed (above) fail in identifying any exoplanet as expected.


### Project design

The workflow for solving this problem would have the following order:

    1) Exploring the datasets (check dimensions of data, labels, etc.)
    2) Data preprocessing: 
    2.1) The datasets provided by kaggle are supposed to be clean (for the most part).
    2.2) It is noted that the data is not normalized. (Some normalization is required.)
    2.3) Github reference (above) mentions that techniques like data augmentation could help as we are dealing with time series. Perhaps systematically shifting rows and adding noise could generate additional realistic (albeit synthetic) trends. This might be explored depending on results from the different algorithms tested for solving the problem.
    3) Evaluate machine learning algorithms: This involves building the models and selecting best model by using evaluation metrics and comparing to benchmark model(s).
    4) Model tuning to optimize results: This involves using evaluating performance of the model and fine tuning hyper-parameters until a satisfactory model is identified. This is by using evaluation metrics to evaluate performance.
    5) Final conclusions.


In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import itertools
from IPython.display import display # Allows the use of display() for DataFrames
from get_results import plot_roc_auc, confusion_matrix_com

# Pretty display for notebooks
%matplotlib inline

# Some Sklearn libraries are required
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils.fixes import signature
from sklearn.metrics import average_precision_score

In [2]:
# Load the training data from Exoplanet dataset
train_data = pd.read_csv('kepler/exoTrain.csv')

In [3]:
# Find dimensions of the train data
train_data.shape

(5087, 3198)

In [4]:
# Testing  if train_data has any null fields
testing = pd.isnull(train_data)
testing *= 1
testing2 = testing.sum()
testing2.sum() # If results is zero, then there a no fields with "null" value

0

In [5]:
# Getting X_train and y_train
# Using iloc to select data using position instead of label and converting to numpy array using values
X_train = train_data.iloc[:,1:].values 
y_train = train_data.iloc[:,0:1].values 

In [6]:
# Find dimensions of the X_train data
X_train.shape

(5087, 3197)

In [7]:
# Find dimensions of the labels (y_train) data
y_train.shape

(5087, 1)

In [8]:
# y_train: Label is 2 for exoplanet and 1 for non-exoplanet
y_train[:5]

array([[2],
       [2],
       [2],
       [2],
       [2]])

In [9]:
y_train -= 1 # Changing labels to: 1 for exoplanet and 0 for non-exoplanet

In [10]:
y_train[:5]

array([[1],
       [1],
       [1],
       [1],
       [1]])

In [11]:
# Load the testing data from Exoplanet dataset
test_data = pd.read_csv('kepler/exoTest.csv')

In [12]:
# Find dimensions of the test data
test_data.shape

(570, 3198)

In [13]:
# Testing  if test_data has any "null" fields
testing = pd.isnull(test_data)
testing *= 1
testing2 = testing.sum()
testing2.sum() # If results is zero, then there a no fields with "null" value

0

In [14]:
# Getting X_test and y_test
# Using iloc to select data using position instead of label and converting to numpy array (using values)
X_test = test_data.iloc[:,1:].values
y_test = test_data.iloc[:,0:1].values

In [15]:
# Find dimensions of the X_test data
X_test.shape

(570, 3197)

In [16]:
# Find dimensions of the labels (y_test) data
y_test.shape

(570, 1)

In [17]:
# y_test: Label is 2 for exoplanet and 1 for non-exoplanet
y_test[:6]

array([[2],
       [2],
       [2],
       [2],
       [2],
       [1]])

In [18]:
y_test -= 1 # Changing labels to: 1 for exoplanet and 0 for non-exoplanet

In [19]:
y_test[:6]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [0]])

In [20]:
# Normalizing the data since it is not normalized according to Kaggle/Github
from sklearn.preprocessing import StandardScaler

In [21]:
# Checking X_train data
X_train

array([[  93.85,   83.81,   20.1 , ...,   61.42,    5.08,  -39.54],
       [ -38.88,  -33.83,  -58.54, ...,    6.46,   16.  ,   19.93],
       [ 532.64,  535.92,  513.73, ...,  -28.91,  -70.02,  -96.67],
       ..., 
       [ 273.39,  278.  ,  261.73, ...,   88.42,   79.07,   79.43],
       [   3.82,    2.09,   -3.29, ...,  -14.55,   -6.41,   -2.55],
       [ 323.28,  306.36,  293.16, ...,  -16.72,  -14.09,   27.82]])

In [22]:
# Tranposing X_train before applying scaling such that mean is zero and variance is one
X_train.transpose()

array([[  93.85,  -38.88,  532.64, ...,  273.39,    3.82,  323.28],
       [  83.81,  -33.83,  535.92, ...,  278.  ,    2.09,  306.36],
       [  20.1 ,  -58.54,  513.73, ...,  261.73,   -3.29,  293.16],
       ..., 
       [  61.42,    6.46,  -28.91, ...,   88.42,  -14.55,  -16.72],
       [   5.08,   16.  ,  -70.02, ...,   79.07,   -6.41,  -14.09],
       [ -39.54,   19.93,  -96.67, ...,   79.43,   -2.55,   27.82]])

In [23]:
# # Checking X_train data
X_test

array([[  1.19880000e+02,   1.00210000e+02,   8.64600000e+01, ...,
          3.57800000e+01,   2.69430000e+02,   5.77200000e+01],
       [  5.73659000e+03,   5.69998000e+03,   5.71716000e+03, ...,
         -2.36619000e+03,  -2.29486000e+03,  -2.03472000e+03],
       [  8.44480000e+02,   8.17490000e+02,   7.70070000e+02, ...,
         -1.62680000e+02,  -3.67900000e+01,   3.06300000e+01],
       ..., 
       [ -5.40100000e+01,  -4.41300000e+01,  -4.12300000e+01, ...,
          5.47000000e+00,   1.44600000e+01,   1.87000000e+01],
       [  9.13600000e+01,   8.56000000e+01,   4.88100000e+01, ...,
         -8.43000000e+00,  -6.48000000e+00,   1.76000000e+01],
       [  3.07119000e+03,   2.78253000e+03,   2.60869000e+03, ...,
         -2.77220000e+02,  -6.96300000e+01,   1.21560000e+02]])

In [24]:
# Tranposing X_test before applying scaling such that mean is zero and variance is one
X_test.transpose()

array([[  1.19880000e+02,   5.73659000e+03,   8.44480000e+02, ...,
         -5.40100000e+01,   9.13600000e+01,   3.07119000e+03],
       [  1.00210000e+02,   5.69998000e+03,   8.17490000e+02, ...,
         -4.41300000e+01,   8.56000000e+01,   2.78253000e+03],
       [  8.64600000e+01,   5.71716000e+03,   7.70070000e+02, ...,
         -4.12300000e+01,   4.88100000e+01,   2.60869000e+03],
       ..., 
       [  3.57800000e+01,  -2.36619000e+03,  -1.62680000e+02, ...,
          5.47000000e+00,  -8.43000000e+00,  -2.77220000e+02],
       [  2.69430000e+02,  -2.29486000e+03,  -3.67900000e+01, ...,
          1.44600000e+01,  -6.48000000e+00,  -6.96300000e+01],
       [  5.77200000e+01,  -2.03472000e+03,   3.06300000e+01, ...,
          1.87000000e+01,   1.76000000e+01,   1.21560000e+02]])

In [25]:
# Standardize features by removing the mean and scaling to unit variance (sklearn.preprocessing)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # Output in a numpy.ndarray
X_test = scaler.fit_transform(X_test) # Output in a numpy.ndarray

In [26]:
X_train.transpose() # Transpose back to original dimensions

array([[-0.00235557, -0.00852774,  0.01804893, ...,  0.00599336,
        -0.00654212,  0.00831333],
       [-0.00205404, -0.0074516 ,  0.01868969, ...,  0.00685579,
        -0.00580352,  0.00815701],
       [-0.00579778, -0.00938685,  0.01673115, ...,  0.00523005,
        -0.00686528,  0.00666449],
       ..., 
       [ 0.0341983 ,  0.03109682,  0.02910084, ...,  0.03572195,
         0.0299112 ,  0.02978874],
       [ 0.02736753,  0.02803863,  0.02275218, ...,  0.03191466,
         0.0266614 ,  0.02618942],
       [ 0.01805157,  0.02216476,  0.01410023, ...,  0.02628002,
         0.02060995,  0.02271046]])

In [27]:
X_test.transpose() # Transpose back to original dimensions

array([[-0.03143654,  0.41497541,  0.02615413, ..., -0.04525719,
        -0.03370329,  0.20313139],
       [-0.05057432,  0.39343385,  0.00629912, ..., -0.0620191 ,
        -0.05173276,  0.16210798],
       [-0.03559448,  0.41363759,  0.01894572, ..., -0.04578193,
        -0.0385983 ,  0.16563567],
       ..., 
       [-0.00907555, -0.2624662 , -0.03001166, ..., -0.01227304,
        -0.01373939, -0.04209481],
       [ 0.00470934, -0.26591026, -0.02760726, ..., -0.02219864,
        -0.02440852, -0.03107299],
       [-0.00786554, -0.22375419, -0.01066057, ..., -0.01189145,
        -0.01200494, -0.00127881]])

In [28]:
# Fix random seed for reproducibility
seed = 10
np.random.seed(seed)

In [29]:
# Importing Keras libraries

from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPool1D, Dense, Dropout, Flatten
from keras.layers import BatchNormalization, Input, concatenate, Activation
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from keras.callbacks import ModelCheckpoint 

Using TensorFlow backend.


In [30]:
#Convert data into 3d tensor (Input 0 in Conv1D is incompatible with layer conv1d_1: expected ndim=3, found ndim=2)
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [31]:
# Checking shape of X_test tensor
X_test.shape

(570, 3197, 1)

In [32]:
# Checking shape of X_train_new tensor
X_train.shape

(5087, 3197, 1)

#### Data exploration and preparation ended (above)

# Ending Data Preparation

# GRID SEARCH 

## Grid Search Implementation

Reference on using GridSearch for tuning Hyperparameters for DL models:

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

# GRID SEARCH 3cnn, 2dnn

In [33]:
# Selecting a subset of the training set
X_GS1000 = X_train[0:1000,:]
y_GS1000 = y_train[0:1000,:]

In [34]:
# Use scikit-learn to grid search the batch size and epochs
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model_drop(optimizer='adam', dropout_rate=0.0):
    # create model
    model = Sequential()
    # Defining network architecture
    model.add(Conv1D(filters=8, kernel_size=8, activation='relu', input_shape=(3197,1)))
    model.add(MaxPool1D(strides=4))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16, kernel_size=8, activation='relu'))
    model.add(MaxPool1D(strides=4))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
    model.add(MaxPool1D(strides=4))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model
# Fix random seed for reproducibility (done above on previous cells)
# Dataset was created in previous section


In [35]:
# Create model
model = KerasClassifier(build_fn=create_model_drop, epochs=30, batch_size=32, verbose=2)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 25, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_GS1000, y_GS1000)


Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/25
4s - loss: 0.2688 - acc: 0.9475
4s - loss: 0.0146 - acc: 1.0000
Epoch 2/10
Epoch 2/10
4s - loss: 0.2807 - acc: 0.9355
Epoch 2/10
5s - loss: 0.0151 - acc: 1.0000
Epoch 2/25
4s - loss: 0.1883 - acc: 0.9520
Epoch 3/10
4s - loss: 2.2060e-04 - acc: 1.0000
Epoch 3/10
4s - loss: 0.1939 - acc: 0.9505
Epoch 3/10
4s - loss: 2.1109e-04 - acc: 1.0000
Epoch 3/25
4s - loss: 0.1854 - acc: 0.9535
Epoch 4/10
4s - loss: 9.6302e-05 - acc: 1.0000
Epoch 4/10
4s - loss: 0.2375 - acc: 0.9430
Epoch 4/10
4s - loss: 1.0400e-04 - acc: 1.0000
Epoch 4/25
4s - loss: 0.1714 - acc: 0.9535
Epoch 5/10
4s - loss: 6.6581e-05 - acc: 1.0000
Epoch 5/10
4s - loss: 0.1826 - acc: 0.9505
Epoch 5/10
4s - loss: 7.1068e-05 - acc: 1.0000
Epoch 5/25
4s - loss: 0.1394 - acc: 0.9685
Epoch 6/10
4s - loss: 4.4589e-05 - acc: 1.0000
4s - loss: 0.1557 - acc: 0.9595
Epoch 6/10
Epoch 6/10
4s - loss: 4.7085e-05 - acc: 1.0000
Epoch 6/25
4s - loss: 0.1217 - acc: 0.9670
Epoch 7/10
4s - loss: 3.6850e-05

Epoch 11/50
4s - loss: 5.1597e-05 - acc: 1.0000
Epoch 11/100
4s - loss: 6.5314e-06 - acc: 1.0000
Epoch 38/50
4s - loss: 0.0867 - acc: 0.9745
Epoch 23/50
4s - loss: 0.1119 - acc: 0.9760
Epoch 12/50
4s - loss: 2.8242e-05 - acc: 1.0000
Epoch 12/100
4s - loss: 3.5764e-06 - acc: 1.0000
Epoch 39/50
4s - loss: 0.0640 - acc: 0.9865
Epoch 24/50
4s - loss: 0.0762 - acc: 0.9790
Epoch 13/50
4s - loss: 2.1005e-05 - acc: 1.0000
Epoch 13/100
4s - loss: 4.1725e-06 - acc: 1.0000
Epoch 40/50
4s - loss: 0.0891 - acc: 0.9760
Epoch 25/50
4s - loss: 0.0693 - acc: 0.9820
Epoch 14/50
4s - loss: 2.2211e-05 - acc: 1.0000
Epoch 14/100
4s - loss: 2.7997e-06 - acc: 1.0000
Epoch 41/50
4s - loss: 0.1049 - acc: 0.9760
Epoch 26/50
4s - loss: 0.0625 - acc: 0.9820
Epoch 15/50
4s - loss: 1.7149e-05 - acc: 1.0000
Epoch 15/100
4s - loss: 3.1301e-06 - acc: 1.0000
Epoch 42/50
4s - loss: 0.0452 - acc: 0.9880
Epoch 27/50
4s - loss: 0.0910 - acc: 0.9670
Epoch 16/50
4s - loss: 1.0359e-05 - acc: 1.0000
Epoch 16/100
4s - loss: 3.0

Epoch 57/100
4s - loss: 0.0596 - acc: 0.9850
Epoch 32/100
4s - loss: 0.0731 - acc: 0.9850
Epoch 17/100
4s - loss: 3.7013e-04 - acc: 1.0000
Epoch 6/10
4s - loss: 6.9631e-07 - acc: 1.0000
Epoch 58/100
4s - loss: 0.0406 - acc: 0.9895
Epoch 33/100
4s - loss: 0.0981 - acc: 0.9760
Epoch 18/100
4s - loss: 2.8809e-04 - acc: 1.0000
Epoch 7/10
4s - loss: 7.2150e-07 - acc: 1.0000
Epoch 59/100
4s - loss: 0.0474 - acc: 0.9895
Epoch 34/100
4s - loss: 0.1695 - acc: 0.9700
Epoch 19/100
4s - loss: 2.4461e-04 - acc: 1.0000
Epoch 8/10
4s - loss: 8.8705e-07 - acc: 1.0000
Epoch 60/100
4s - loss: 0.0429 - acc: 0.9850
Epoch 35/100
3s - loss: 1.1529e-04 - acc: 1.0000
Epoch 9/10
4s - loss: 0.0904 - acc: 0.9775
Epoch 20/100
4s - loss: 8.7846e-07 - acc: 1.0000
Epoch 61/100
4s - loss: 0.0476 - acc: 0.9880
Epoch 36/100
4s - loss: 1.6396e-04 - acc: 1.0000
Epoch 10/10
4s - loss: 0.0786 - acc: 0.9835
Epoch 21/100
4s - loss: 8.3446e-07 - acc: 1.0000
Epoch 62/100
4s - loss: 1.2376e-04 - acc: 1.0000
4s - loss: 0.0438 - 

Epoch 63/100
5s - loss: 0.3665 - acc: 0.8966
Epoch 2/25
4s - loss: 2.5023e-04 - acc: 1.0000
Epoch 17/25
4s - loss: 0.0013 - acc: 1.0000
Epoch 79/100
4s - loss: 0.0024 - acc: 1.0000
Epoch 64/100
4s - loss: 0.2024 - acc: 0.9460
Epoch 3/25
4s - loss: 1.4011e-04 - acc: 1.0000
Epoch 18/25
4s - loss: 0.0268 - acc: 0.9940
Epoch 80/100
4s - loss: 0.0144 - acc: 0.9955
Epoch 65/100
4s - loss: 0.1915 - acc: 0.9475
Epoch 4/25
4s - loss: 1.3282e-04 - acc: 1.0000
Epoch 19/25
4s - loss: 0.0209 - acc: 0.9910
Epoch 81/100
4s - loss: 0.0069 - acc: 0.9985
Epoch 66/100
4s - loss: 0.1937 - acc: 0.9505
Epoch 5/25
4s - loss: 1.5478e-04 - acc: 1.0000
Epoch 20/25
4s - loss: 0.0072 - acc: 0.9985
Epoch 82/100
4s - loss: 0.0014 - acc: 1.0000
Epoch 67/100
4s - loss: 0.1671 - acc: 0.9595
Epoch 6/25
3s - loss: 1.3574e-04 - acc: 1.0000
Epoch 21/25
4s - loss: 0.0197 - acc: 0.9910
Epoch 83/100
4s - loss: 0.0071 - acc: 0.9970
Epoch 68/100
4s - loss: 0.1666 - acc: 0.9580
Epoch 7/25
4s - loss: 1.0447e-04 - acc: 1.0000
Epo

3s - loss: 0.1577 - acc: 0.9640
Epoch 11/50
4s - loss: 0.0945 - acc: 0.9805
Epoch 24/50
4s - loss: 1.0433e-04 - acc: 1.0000
Epoch 9/100
4s - loss: 2.9530e-05 - acc: 1.0000
Epoch 25/50
4s - loss: 0.1140 - acc: 0.9745
Epoch 12/50
4s - loss: 0.0897 - acc: 0.9760
Epoch 25/50
3s - loss: 6.6123e-05 - acc: 1.0000
Epoch 10/100
4s - loss: 1.9106e-05 - acc: 1.0000
Epoch 26/50
4s - loss: 0.1002 - acc: 0.9715
Epoch 13/50
4s - loss: 0.0947 - acc: 0.9775
Epoch 26/50
4s - loss: 3.9428e-05 - acc: 1.0000
Epoch 11/100
4s - loss: 2.1137e-05 - acc: 1.0000
Epoch 27/50
3s - loss: 0.0997 - acc: 0.9775
Epoch 14/50
4s - loss: 0.0798 - acc: 0.9790
Epoch 27/50
4s - loss: 3.6375e-05 - acc: 1.0000
Epoch 12/100
4s - loss: 2.7984e-05 - acc: 1.0000
Epoch 28/50
3s - loss: 0.1719 - acc: 0.9655
Epoch 15/50
3s - loss: 0.0650 - acc: 0.9805
Epoch 28/50
4s - loss: 4.1132e-05 - acc: 1.0000
Epoch 13/100
4s - loss: 1.6379e-05 - acc: 1.0000
Epoch 29/50
3s - loss: 0.0994 - acc: 0.9745
Epoch 16/50
4s - loss: 0.0692 - acc: 0.9775


...

Epoch 7/100
3s - loss: 0.1073 - acc: 0.9670
3s - loss: 0.0402 - acc: 0.9910
Epoch 49/50
3s - loss: 2.2239e-05 - acc: 1.0000
Epoch 47/100
3s - loss: 0.2219 - acc: 0.9445
Epoch 8/100
3s - loss: 0.0185 - acc: 0.9985
Epoch 50/50
3s - loss: 3.3631e-05 - acc: 1.0000
Epoch 48/100
3s - loss: 0.1899 - acc: 0.9490
Epoch 9/100
3s - loss: 0.0339 - acc: 0.9880
3s - loss: 1.0814e-05 - acc: 1.0000
Epoch 49/100
Epoch 1/100
3s - loss: 0.2192 - acc: 0.9460
Epoch 10/100
3s - loss: 1.7911e-05 - acc: 1.0000
Epoch 50/100
2s - loss: 0.1702 - acc: 0.9520
Epoch 11/100
Epoch 1/10
3s - loss: 2.8343e-05 - acc: 1.0000
Epoch 51/100
3s - loss: 0.1770 - acc: 0.9535
Epoch 12/100
7s - loss: 0.3366 - acc: 0.9250
Epoch 2/100
3s - loss: 3.2345e-05 - acc: 1.0000
Epoch 52/100
3s - loss: 0.1584 - acc: 0.9625
Epoch 13/100
3s - loss: 0.2311 - acc: 0.9505
Epoch 3/100
6s - loss: 0.3266 - acc: 0.8559
Epoch 2/10
3s - loss: 1.3680e-05 - acc: 1.0000
Epoch 53/100
3s - loss: 0.1461 - acc: 0.9595
Epoch 14/100
3s - loss: 0.2017 - acc: 0

Epoch 4/25
4s - loss: 7.6933e-06 - acc: 1.0000
Epoch 95/100
4s - loss: 0.0381 - acc: 0.9880
Epoch 47/100
3s - loss: 0.0708 - acc: 0.9820
Epoch 57/100
3s - loss: 0.0060 - acc: 1.0000
Epoch 5/25
3s - loss: 3.2660e-06 - acc: 1.0000
Epoch 96/100
3s - loss: 0.0251 - acc: 0.9940
Epoch 48/100
3s - loss: 0.0667 - acc: 0.9865
Epoch 58/100
3s - loss: 0.0048 - acc: 0.9970
Epoch 6/25
3s - loss: 6.8722e-06 - acc: 1.0000
Epoch 97/100
3s - loss: 0.0332 - acc: 0.9895
Epoch 49/100
3s - loss: 0.0742 - acc: 0.9760
Epoch 59/100
3s - loss: 0.0020 - acc: 1.0000
Epoch 7/25
3s - loss: 5.5734e-06 - acc: 1.0000
Epoch 98/100
4s - loss: 0.0272 - acc: 0.9925
Epoch 50/100
3s - loss: 0.0017 - acc: 1.0000
Epoch 8/25
4s - loss: 0.0753 - acc: 0.9820
Epoch 60/100
3s - loss: 9.6010e-06 - acc: 1.0000
Epoch 99/100
3s - loss: 0.0360 - acc: 0.9910
Epoch 51/100
3s - loss: 0.0593 - acc: 0.9805
Epoch 61/100
4s - loss: 0.0015 - acc: 1.0000
Epoch 9/25
3s - loss: 3.1668e-06 - acc: 1.0000
Epoch 100/100
3s - loss: 0.0186 - acc: 0.99

3s - loss: 0.0280 - acc: 0.9910
Epoch 96/100
3s - loss: 0.0907 - acc: 0.9760
Epoch 25/25
4s - loss: 0.2391 - acc: 0.9445
Epoch 3/50
3s - loss: 1.1589e-04 - acc: 1.0000
Epoch 12/50
3s - loss: 0.0295 - acc: 0.9910
Epoch 97/100
3s - loss: 0.0883 - acc: 0.9760
3s - loss: 0.2244 - acc: 0.9445
Epoch 4/50
3s - loss: 6.9891e-05 - acc: 1.0000
Epoch 13/50
3s - loss: 0.0197 - acc: 0.9940
Epoch 98/100
3s - loss: 0.2043 - acc: 0.9460
Epoch 5/50
3s - loss: 5.5387e-05 - acc: 1.0000
Epoch 14/50
3s - loss: 0.0196 - acc: 0.9970
Epoch 99/100
3s - loss: 0.1900 - acc: 0.9445
Epoch 6/50
Epoch 1/50
3s - loss: 1.0141e-04 - acc: 1.0000
Epoch 15/50
3s - loss: 0.0415 - acc: 0.9865
Epoch 100/100
3s - loss: 0.1897 - acc: 0.9460
Epoch 7/50
3s - loss: 3.3865e-05 - acc: 1.0000
Epoch 16/50
3s - loss: 0.0265 - acc: 0.9910
3s - loss: 0.1678 - acc: 0.9520
Epoch 8/50
3s - loss: 6.1125e-05 - acc: 1.0000
Epoch 17/50
7s - loss: 0.4151 - acc: 0.8786
Epoch 2/50
3s - loss: 0.1792 - acc: 0.9490
Epoch 9/50
3s - loss: 5.5310e-05 -

Epoch 44/50
3s - loss: 2.2213e-05 - acc: 1.0000
Epoch 40/100
3s - loss: 0.2125 - acc: 0.9475
Epoch 5/100
4s - loss: 0.0607 - acc: 0.9805
3s - loss: 2.0781e-05 - acc: 1.0000
Epoch 41/100
3s - loss: 0.0257 - acc: 0.9925
Epoch 45/50
3s - loss: 0.1964 - acc: 0.9535
Epoch 6/100
3s - loss: 2.4029e-05 - acc: 1.0000
Epoch 42/100
3s - loss: 0.0315 - acc: 0.9910
Epoch 46/50
3s - loss: 0.1865 - acc: 0.9520
Epoch 7/100
3s - loss: 1.8110e-05 - acc: 1.0000
Epoch 43/100
3s - loss: 0.0198 - acc: 0.9955
Epoch 47/50
3s - loss: 0.1783 - acc: 0.9505
Epoch 8/100
Epoch 1/100
3s - loss: 1.7426e-05 - acc: 1.0000
Epoch 44/100
3s - loss: 0.0158 - acc: 0.9970
Epoch 48/50
3s - loss: 0.2005 - acc: 0.9490
Epoch 9/100
3s - loss: 1.6418e-05 - acc: 1.0000
Epoch 45/100
4s - loss: 0.0196 - acc: 0.9955
Epoch 49/50
3s - loss: 0.1803 - acc: 0.9535
Epoch 10/100
6s - loss: 0.3611 - acc: 0.8531
Epoch 2/100
3s - loss: 3.7861e-05 - acc: 1.0000
Epoch 46/100
3s - loss: 0.0192 - acc: 0.9925
Epoch 50/50
4s - loss: 0.1927 - acc: 0.9

Epoch 98/100
2s - loss: 0.0820 - acc: 0.9730
Epoch 63/100
3s - loss: 0.0487 - acc: 0.9880
Epoch 55/100
3s - loss: 3.3787e-06 - acc: 1.0000
Epoch 99/100
2s - loss: 0.0739 - acc: 0.9745
Epoch 64/100
3s - loss: 0.1641 - acc: 0.9670
Epoch 56/100
3s - loss: 2.3888e-05 - acc: 1.0000
Epoch 100/100
2s - loss: 0.0731 - acc: 0.9805
Epoch 65/100
3s - loss: 0.0747 - acc: 0.9865
Epoch 57/100
2s - loss: 1.2521e-05 - acc: 1.0000
3s - loss: 0.0724 - acc: 0.9760
Epoch 66/100
2s - loss: 0.0456 - acc: 0.9880
Epoch 58/100
2s - loss: 0.0708 - acc: 0.9760
Epoch 67/100
2s - loss: 0.0344 - acc: 0.9880
Epoch 59/100
2s - loss: 0.0649 - acc: 0.9775
Epoch 68/100
2s - loss: 0.0247 - acc: 0.9940
Epoch 60/100
2s - loss: 0.0579 - acc: 0.9805
Epoch 69/100
2s - loss: 0.0267 - acc: 0.9955
Epoch 61/100
2s - loss: 0.0651 - acc: 0.9805
Epoch 70/100
2s - loss: 0.0362 - acc: 0.9865
Epoch 62/100
2s - loss: 0.0682 - acc: 0.9790
Epoch 71/100
2s - loss: 0.0258 - acc: 0.9940
Epoch 63/100
1s - loss: 0.0652 - acc: 0.9805
Epoch 72/1

In [36]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.963000 using {'batch_size': 80, 'epochs': 50}
0.962000 (0.051554) with: {'batch_size': 10, 'epochs': 10}
0.963000 (0.052248) with: {'batch_size': 10, 'epochs': 25}
0.962000 (0.051554) with: {'batch_size': 10, 'epochs': 50}
0.963000 (0.052248) with: {'batch_size': 10, 'epochs': 100}
0.963000 (0.052248) with: {'batch_size': 20, 'epochs': 10}
0.962000 (0.051554) with: {'batch_size': 20, 'epochs': 25}
0.959000 (0.049657) with: {'batch_size': 20, 'epochs': 50}
0.663000 (0.204708) with: {'batch_size': 20, 'epochs': 100}
0.963000 (0.052248) with: {'batch_size': 40, 'epochs': 10}
0.962000 (0.051554) with: {'batch_size': 40, 'epochs': 25}
0.959000 (0.049657) with: {'batch_size': 40, 'epochs': 50}
0.716000 (0.326578) with: {'batch_size': 40, 'epochs': 100}
0.962000 (0.051554) with: {'batch_size': 60, 'epochs': 10}
0.961000 (0.050890) with: {'batch_size': 60, 'epochs': 25}
0.962000 (0.051554) with: {'batch_size': 60, 'epochs': 50}
0.960000 (0.050258) with: {'batch_size': 60, 'epochs': 100

### Best: 0.963000 using {'batch_size': 80, 'epochs': 50}

    0.962000 (0.051554) with: {'batch_size': 10, 'epochs': 10}
    0.963000 (0.052248) with: {'batch_size': 10, 'epochs': 25}
    0.962000 (0.051554) with: {'batch_size': 10, 'epochs': 50}
    0.963000 (0.052248) with: {'batch_size': 10, 'epochs': 100}
    0.963000 (0.052248) with: {'batch_size': 20, 'epochs': 10}
    0.962000 (0.051554) with: {'batch_size': 20, 'epochs': 25}
    0.959000 (0.049657) with: {'batch_size': 20, 'epochs': 50}
    0.663000 (0.204708) with: {'batch_size': 20, 'epochs': 100}
    0.963000 (0.052248) with: {'batch_size': 40, 'epochs': 10}
    0.962000 (0.051554) with: {'batch_size': 40, 'epochs': 25}
    0.959000 (0.049657) with: {'batch_size': 40, 'epochs': 50}
    0.716000 (0.326578) with: {'batch_size': 40, 'epochs': 100}
    0.962000 (0.051554) with: {'batch_size': 60, 'epochs': 10}
    0.961000 (0.050890) with: {'batch_size': 60, 'epochs': 25}
    0.962000 (0.051554) with: {'batch_size': 60, 'epochs': 50}
    0.960000 (0.050258) with: {'batch_size': 60, 'epochs': 100}
    0.962000 (0.051554) with: {'batch_size': 80, 'epochs': 10}
    0.962000 (0.051554) with: {'batch_size': 80, 'epochs': 25}
    0.963000 (0.052248) with: {'batch_size': 80, 'epochs': 50}
    0.961000 (0.050890) with: {'batch_size': 80, 'epochs': 100}
    0.962000 (0.051554) with: {'batch_size': 100, 'epochs': 10}
    0.962000 (0.051554) with: {'batch_size': 100, 'epochs': 25}
    0.962000 (0.051554) with: {'batch_size': 100, 'epochs': 50}
    0.961000 (0.050890) with: {'batch_size': 100, 'epochs': 100}
